In [33]:
import pandas as pd
import requests
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
import re
import numpy as np
from components.scripts.load_data import load_data
from components.scripts.scraper import google
from model import clean_chop, clean


In [12]:
blacklist = stopwords.words('english')
blacklist += ['card', 'aus', 'au', 'ns', 'nsw', 'xx', 'pty', 'ltd', 'nswau']

tr_data = load_data("CSVData.csv")
tr_data["desc_corpus"] = [clean_chop(desc, blacklist) for desc in tr_data.description]
tr_data["desc_features"] = [clean(desc, blacklist) for desc in tr_data.description]

to_scrape = tr_data[["desc_features"]].drop_duplicates()
to_scrape["google"] = [google(query) for query in to_scrape.desc_features]
# to_scrape["category"] = [categorise(goog) for goog in to_scrape.google]
# to_scrape = to_scrape.drop("google", axis=1)



✅ query: hfm boronia park result: Farmers' market
✅ query: charlotte cafe gladesville result: Cafe
✅ query: glen garry castle ho redfern result: Pub
❌ couldn't find: transfer mr tryone kenhong jabez ja credit account
✅ query: dymocks lane cove result: Book store
✅ query: sq wok passion lane cove result: Asian restaurant
✅ query: pharmacy less ryde result: Drug store
✅ query: function pcp gladesville result: Physiotherapist
❌ couldn't find: transportfornsw opal chippendale
✅ query: werri beach fish shop werri result: Fish and chips takeaway
❌ couldn't find: transfer catharine dodds commbank app scripts
❌ couldn't find: transfer mrs helen andrea dodds credit account
✅ query: newsxpress gerringong result: Newsstand
❌ couldn't find: willoughby city counci chatswood
❌ couldn't find: apple online sydney
✅ query: hungry jacks haywards bay result: Fast food restaurant
❌ couldn't find: mosman iga mosman
❌ couldn't find: transfer miss charlotte alice mcgui credit account
✅ query: dan murphys mos

❌ couldn't find: hungry jacks woolloomoolooau
✅ query: mcdonalds bomaderry result: Fast food restaurant
✅ query: royal george ivy sydney result: Pub
✅ query: orient hotel sydney result: Pub
✅ query: fern st espresso gerringong result: Cafe
✅ query: il locale gelato berry result: Cafe
✅ query: penzo pizza ge nowra result: Restaurant
✅ query: shell berry result: Petrol Station
❌ couldn't find: ivy coffee shop sydney
❌ couldn't find: doordash grilld melbourne vi
✅ query: sushi family sydney result: Sushi restaurant
✅ query: merchants ultimo broadway result: Restaurant
❌ couldn't find: pline ph george sydney
❌ couldn't find: transfer mcguirec credit account
✅ query: thai la ong newtown result: Thai restaurant
✅ query: fernery mosman result: Australian restaurant
✅ query: hfm mosman result: Greengrocer
✅ query: double brew sydney result: Cafe
❌ couldn't find: transfer bank commbank app owings
✅ query: bathers pavilion mosman result: Modern Australian restaurant
❌ couldn't find: bp mosman
❌ 

In [15]:
to_scrape.to_csv("google.csv", index=False)

In [35]:
def google(query:str, verbose=True) -> str:
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    req = requests.get("https://www.google.com/search?q="+query, headers=headers)
    req.raise_for_status()
    soup = BeautifulSoup(req.text, "html.parser")
    bus = soup.find_all("span", class_="YhemCb")  # potential results for business
    comp = soup.find_all("div", class_="wwUB2c PZPZlf") # potential results for larger company
    if len(bus) > 0:
        bus = str(bus[-1])
        bus = bus.split(">")[1].split("<")[0]
        if " in " in bus:
            bus = bus.split(" in ")[0]
        print("✅ query: " + query, "result: " + bus) if verbose else None
        return bus
    elif len(comp) == 1:
            comp = str(comp[0])
            comp = comp.split("</span>")[0].split(">")[-1]
            print("✅ query: " + query, "result (company): " + comp) if verbose else None
            return comp
    else:
        print("❌ couldn't find: " + query) if verbose else None
        return "-"



results = pd.read_csv("google.csv")
results

googled = tr_data.merge(results[5::], how="left", on="desc_features", validate="many_to_one")

ungoogled = googled[pd.isna(googled.google)]

ungoogled = ungoogled[["desc_features"]].drop_duplicates()

ungoogled["google"] = [google(query) for query in ungoogled.desc_features]

ungoogled.to_csv("google.csv", mode = 'a', header = False, index=False)



✅ query: hfm boronia park result: Farmers' market
✅ query: charlotte cafe gladesville result: Cafe
✅ query: glen garry castle ho redfern result: Pub
❌ couldn't find: transfer mr tryone kenhong jabez ja credit account
✅ query: dymocks lane cove result: Book store


,desc_features,google
0,hfm boronia park,Farmers' market
1,charlotte cafe gladesville,Cafe
2,glen garry castle ho redfern,Pub
4,transfer mr tryone kenhong jabez ja credit acc...,-
5,dymocks lane cove,Book store


In [52]:
blacklist = stopwords.words('english')
blacklist += ['card', 'aus', 'au', 'ns', 'nsw', 'xx']

def clean(string):
    features = ""
    out = string
    if "value date: " in out:
        out = out.split("value date: ")[0]
    out = re.sub(r'([^a-z ]+)', " ", out) # remove non-alphabetic characters
    for word in out.strip().split(" "):
        if word not in blacklist and len(word) > 1 and " " + word + " " not in features: # ensuring no duplicates
            features += word + " "
    return features.strip()

data["desc_features"] = [clean(desc) for desc in data.description]
data

,date,amount,description,desc_features
0,2020-07-10,-6.99,hfm boronia park boronia park au card ...,hfm boronia park
1,2020-07-10,-8.40,charlotte cafe gladesville au,charlotte cafe gladesville
2,2020-07-08,-12.00,glen garry castle ho redfern au card ...,glen garry castle ho redfern
3,2020-07-08,-12.00,glen garry castle ho redfern au card ...,glen garry castle ho redfern
4,2020-07-08,15.00,transfer from mr tryone kenhong jabez ja credi...,transfer mr tryone kenhong jabez ja credit acc...
...,...,...,...,...
595,2019-09-04,-17.00,mr yeeros newtown flinders view ns aus card xx...,mr yeeros newtown flinders view
596,2019-09-04,-12.99,7-eleven 2118 camperdown ns aus card xx6725 va...,eleven camperdown
597,2019-09-04,-2.50,coles express 1544 drummoyne au aus card xx672...,coles express drummoyne
598,2019-09-03,-8.99,qe foodstores camperdown au,qe foodstores camperdown


In [14]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

def google(query):
    req = requests.get("https://www.google.com/search?q="+query, headers=headers)
    req.raise_for_status()
    soup = BeautifulSoup(req.text, "html.parser")
    cat = soup.find_all("span", class_="YhemCb")
    if len(cat) > 0:
        cat = str(cat[-1])
        cat = cat.split(">")[1].split("<")[0]
        if " in " in cat:
            cat = cat.split(" in ")[0]
        print(query, cat)
        return cat
    else:
        print(query)
        return None

    

data["google"] = [google(query) for query in data.desc_features]
data



    

hfm boronia park Farmers' market
charlotte cafe gladesville Cafe
glen garry castle ho redfern Pub
glen garry castle ho redfern Pub
transfer mr tryone kenhong jabez ja credit account
dymocks lane cove Book store
sq wok passion lane cove Asian restaurant
transfer mr tryone kenhong jabez ja credit account
pharmacy less ryde Drug store
function pcp gladesville Physiotherapist
transportfornsw opal chippendale
werri beach fish shop werri Fish and chips takeaway
transfer catharine dodds commbank app scripts
transfer mrs helen andrea dodds credit account
hfm boronia park Farmers' market
newsxpress gerringong Newsstand
willoughby city counci chatswood
apple online sydney
hungry jacks haywards bay Fast food restaurant
mosman iga mosman
transfer miss charlotte alice mcgui credit account
dan murphys mosman Bottle Shop and Liquor Store
mosman iga mosman
transfer commbank app laptop
function pcp gladesville Physiotherapist
charlotte cafe gladesville Cafe
transfer margaret jorgensen izzy tutoring
ele

add thai pty ltd sydney
mondaytosunday pty mosman
haps hotels pty ltd gladesville
brasserie hunters hill Cafe
transfer jack holdsworth commbank app birthday money
destination roll pty sydney Restaurant
illuminated holdings sydney
sfs sydney Stadium
sfs sydney Stadium
newslink syd sydney
transportfornsw opal chippendale
cafe mosman
destination roll pty sydney Restaurant
mcdonalds strand sydney nswau Fast food restaurant
aldi stores gladesvi gladesville Grocery store
chargrill mosman Chicken shop
scarborough hotel sc scarborough Pub
scarborough hotel sc scarborough Pub
scarborough hotel sc scarborough Pub
scarborough hotel sc scarborough Pub
scarborough hotel sc scarborough Pub
sfs sydney Stadium
transportfornsw opal chippendale
criteron hotel sydney Irish pub
hungry jacks woolloomoolooau
mcdonalds bomaderry Fast food restaurant
famous berry Restaurant
transportfornsw opal chippendale
destination roll pty sydney Restaurant
royal george ivy sydney Pub
royal george ivy sydney Pub
orient ho

taste baguette universit Cafe
ralph cafe universit
franks pizza bar camperdown Pizza restaurant
sq dail camperdown
sq dail camperdown
uberuae eats sydney
eleven camperdown
uberuae eats sydney
grounded cafe camperdown Cafe
coles express hunters hill Convenience store
eastern suburbs dist bellevue hill
eastern suburbs dist bellevue hill
eastern suburbs dist bellevue hill
transportfornsw opal chippendale
uberuae eats sydney
uberuae eats sydney
eleven camperdown
eleven camperdown
uber trip sydney
grounded cafe camperdown Cafe
grounded cafe camperdown Cafe
debit excess interest
ralph cafe universit
delish pork pl newtown Take Away Restaurant
transfer anne marie brown tutoring maths
eleven camperdown
eleven camperdown
alfred hotel mngmnt camperdown Pub
uberuae eats sydney
taste baguette universit Cafe
mcdonalds stanmore nswau Fast food restaurant
eleven camperdown
eleven camperdown
eleven camperdown
uber eats sydney
uber eats sydney
uber eats sydney
officeworks glebe Office supply store
offi

,date,amount,description,desc_features,google
0,2020-07-10,-6.99,hfm boronia park boronia park au card ...,hfm boronia park,Farmers' market
1,2020-07-10,-8.40,charlotte cafe gladesville au,charlotte cafe gladesville,Cafe
2,2020-07-08,-12.00,glen garry castle ho redfern au card ...,glen garry castle ho redfern,Pub
3,2020-07-08,-12.00,glen garry castle ho redfern au card ...,glen garry castle ho redfern,Pub
4,2020-07-08,15.00,transfer from mr tryone kenhong jabez ja credi...,transfer mr tryone kenhong jabez ja credit acc...,None
...,...,...,...,...,...
595,2019-09-04,-17.00,mr yeeros newtown flinders view ns aus card xx...,mr yeeros newtown flinders view,Fast food restaurant
596,2019-09-04,-12.99,7-eleven 2118 camperdown ns aus card xx6725 va...,eleven camperdown,None
597,2019-09-04,-2.50,coles express 1544 drummoyne au aus card xx672...,coles express drummoyne,None
598,2019-09-03,-8.99,qe foodstores camperdown au,qe foodstores camperdown,Convenience store


In [15]:
data.to_csv("scraper.csv")

In [53]:
scraper = pd.read_csv("scraper.csv", index_col=0)


In [54]:


def categorise(goog):
    shopping = ["market", "store", "shop"]
    food = ["restaurant", "cafe", "bakery", "takeaway"]
    beers = ["bar", "pub", "hotel"]
    health = ["dentist", "physiotherapist", "drug", "pharmacy", "pharmacist"]
    transport = ["petrol", "gas"]
    
    if pd.isnull(goog):
        return goog
    goog = goog.lower()

    for feat in health:
        if feat in goog:
            return "life/wellbeing"
    for feat in food:
        if feat in goog:
            return "food"
    for feat in beers:
        if feat in goog:
            return "beers"
    for feat in transport:
        if feat in goog:
            return "transport"
    for feat in shopping:
        if feat in goog:
            return "shopping"
    return None
    
    
scraper["category"] = [categorise(goog) for goog in scraper.google]
scraper = scraper.drop("google", axis=1)
    

In [119]:
labels = pd.read_csv("transactions_labelled.csv", index_col=0)
labels["desc_features"] = [clean(desc) for desc in labels.description]
labels

print(len(labels[["desc_features"]].drop_duplicates()))
print(len(labels[["desc_features", "category"]].drop_duplicates()))
labels


250
253


,date,amount,description,category,desc_features
1,2020-06-26,50.00,transfer from margaret a jorgensen izzy tutoring,wages,transfer margaret jorgensen izzy tutoring
2,2020-06-08,50.00,transfer from margaret a jorgensen izzy tutoring,wages,transfer margaret jorgensen izzy tutoring
3,2020-05-25,50.00,transfer from margaret a jorgensen izzy tutoring,wages,transfer margaret jorgensen izzy tutoring
4,2020-05-18,50.00,transfer from margaret a jorgensen izzy tutoring,wages,transfer margaret jorgensen izzy tutoring
5,2020-05-03,50.00,transfer from margaret a jorgensen izzy tutoring,wages,transfer margaret jorgensen izzy tutoring
...,...,...,...,...,...
595,2019-08-28,-6.10,uber *trip sydney ns aus card xx6725 value d...,uber,uber trip sydney
596,2019-08-28,-16.84,uber *trip sydney ns aus card xx6725 value d...,uber,uber trip sydney
597,2019-08-28,-10.00,sq *an dail camperdown au aus card xx6725 valu...,beers,sq dail camperdown
598,2019-08-27,-9.00,mcdonalds stanmore stanmore nswau,food,mcdonalds stanmore nswau


In [123]:
# data["category"] = [None for _ in range(10)] + list(scraper.category[10::])
# data.merge(scraper.iloc[[1, 3, 5, 6]], left_index=True, right_index=True, how="left")
test = data.iloc[30:110]
print(len(test))
print(len(labels))

# print(len(test.merge(train, on=["desc_features", "amount"], how="left")))


lookup = pd.DataFrame(labels.drop(["date", 'amount','description'],axis=1).groupby(["desc_features"])["category"].unique().apply(','.join))
lookup = lookup.reset_index()
lookup = lookup[~lookup.category.str.contains(",")]
test.merge(lookup, on="desc_features", how="left", validate="many_to_one")



80
573


,date,amount,description,desc_features,category
0,2020-06-25,-7.50,t famous berry don v berry aus card xx6725 va...,famous berry,food
1,2020-06-24,-16.00,haps hotels pty ltd gladesville ns aus card xx...,haps hotels pty ltd gladesville,beers
2,2020-06-24,-8.00,haps hotels pty ltd gladesville ns aus card xx...,haps hotels pty ltd gladesville,beers
3,2020-06-24,-7.40,haps hotels pty ltd gladesville ns aus card xx...,haps hotels pty ltd gladesville,beers
4,2020-06-24,-40.10,doitao thai boronia boronia park ns aus card x...,doitao thai boronia park,NaN
...,...,...,...,...,...
75,2020-03-29,50.00,transfer from margaret a jorgensen izzy tutoring,transfer margaret jorgensen izzy tutoring,wages
76,2020-03-28,150.00,transfer from john holdsworth credit to account,transfer john holdsworth credit account,transfer
77,2020-03-26,50.00,direct credit 128594 hartley brady 1hr maths,direct credit hartley brady hr maths,wages
78,2020-03-26,-194.22,kogan.com chffd6y9 melbourne au aus card xx672...,kogan com chffd melbourne,shopping


In [128]:

np.linspace(0,0.1,11)

array([0.  , 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 ])